<a href="https://colab.research.google.com/github/mishkajethwani/summer-research-24/blob/main/Actual_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Important Installations
! pip install --quiet langchain
! pip install -qU --quiet langchain-openai
! pip install --quiet langchain_community
! pip install --quiet -qU langchain-openai
! pip install --upgrade --quiet langchain langchainhub
! pip install --upgrade --quiet  wikipedia
! pip install yake

In [4]:
#Setting the API key
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

··········


In [5]:
# Function to get History

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
# All tools are defined here

from langchain_core.tools import tool

#Tools for Calculator

@tool
def subtract(integer_1: int, integer_2: int) -> int:
    """Subtract integer_2 from integer_1."""
    return integer_1 - integer_2

@tool
def multiply(integer_1: int, integer_2: int) -> int:
    """Multiply two integers."""
    return integer_1 * integer_2

@tool
def divide(integer_1: float, integer_2: float) -> float:
    """Divide integer_1 by integer_2. Returns a float."""
    return integer_1 / integer_2

@tool
def modulo(integer_1: int, integer_2: int) -> int:
    """Calculate the remainder of division."""
    return integer_1 % integer_2


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

@tool
def factorial(integer_1: int) -> int:
    """Calculate the factorial of a non-negative integer."""
    result = 1
    for i in range(2, integer_1 + 1):
        result *= i
    return result

# Tool for wikipedia
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

def wiki(input):
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    return wikipedia.run(input)

chat_tools = [subtract, multiply, divide, modulo, factorial, add, exponentiate, wiki]

In [19]:
# Run for Basic Chat bot with history
from langchain_core.messages import HumanMessage, AIMessage

def chat_bot_normal():
    session_id = "abc12"
    config = {"configurable": {"session_id": session_id}}

    print("Chatbot: Hello! How can I assist you today?")

    while True:
        user_input = input("You: ")
        messages = [HumanMessage(content=user_input)]
        response = with_message_history.invoke(messages, config)
        bot_response = response.content


        print("Chatbot:", bot_response)
chat_bot_normal()

Chatbot: Hello! How can I assist you today?
You: what is python
Chatbot: Python is a high-level programming language known for its simplicity and readability. It is widely used in various fields such as web development, data science, machine learning, artificial intelligence, and more. Python has a large standard library that provides support for a wide range of programming tasks and is also known for its extensive community support and documentation.


KeyboardInterrupt: Interrupted by user

In [ ]:
# Run for Chat bot with a calculator
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory

tools = [subtract, multiply, divide, modulo, factorial, add, exponentiate]

def setup_math_agent():
    llm = ChatOpenAI(temperature=0)

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a precise mathematical calculator. Follow these steps for each calculation:
        1. Break down the problem into individual operations.
        2. Perform each operation step-by-step, using the appropriate tool for each.
        3. Show your work clearly, stating which tool you're using and the numbers you're inputting.
        4. Double-check your work and provide the final answer.
        Available tools: subtract, multiply, divide, modulo, factorial, add, exponentiate"""),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    agent = create_openai_tools_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

def chatbot_updated_calculator():
    llm = ChatOpenAI(temperature=0)
    math_agent = setup_math_agent()

    math_tool = Tool(
        name="math_calculator",
        func=lambda x: math_agent.invoke({"input": x})['output'],
        description="Performs complex mathematical calculations. Input should be a mathematical problem."
    )
    chatbot_tools = [math_tool]
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    chatbot_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful assistant capable of general conversation and mathematical calculations.
        For mathematical problems, use the math_calculator tool. For all other queries, respond normally.
        Always maintain context from previous messages and use that information in your responses.
        If you're not sure about something, admit that you don't know or don't have that information."""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    chatbot_agent = create_openai_tools_agent(llm, chatbot_tools, chatbot_prompt)
    chatbot_executor = AgentExecutor(agent=chatbot_agent, tools=chatbot_tools, memory=memory, verbose=True)

    print("Chatbot: Hello! I'm a chatbot that can also do mathematical calculations. How can I assist you today?")

    while True:
        user_input = input("You: ")
        result = chatbot_executor.invoke({"input": user_input})
        bot_response = result['output']

chatbot_updated_calculator()

In [18]:
# Run for chatbot with History+Math Tools+ RAG with wiki
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
import yake

tools = [subtract, multiply, divide, modulo, factorial, add, exponentiate]

def setup_math_agent():
    llm = ChatOpenAI(temperature=0)

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a precise mathematical calculator. Follow these steps for each calculation:
        1. Break down the problem into individual operations.
        2. Perform each operation step-by-step, using the appropriate tool for each.
        3. Show your work clearly, stating which tool you're using and the numbers you're inputting.
        4. Double-check your work and provide the final answer.
        Available tools: subtract, multiply, divide, modulo, factorial, add, exponentiate"""),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    agent = create_openai_tools_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=True)

def extract_keywords(text):
    language = "en"
    max_ngram_size = 2
    deduplication_threshold = 0.9
    numOfKeywords = 5
    custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
    keywords = custom_kw_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords]

def chatbot_updated_wiki():
    llm = ChatOpenAI(temperature=0)
    math_agent = setup_math_agent()

    math_tool = Tool(
        name="math_calculator",
        func=lambda x: math_agent.invoke({"input": x})['output'],
        description="Performs complex mathematical calculations. Input should be a mathematical problem."
    )
    chatbot_tools = [math_tool]
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    chatbot_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a helpful assistant capable of general conversation and mathematical calculations.
        For mathematical problems, use the math_calculator tool. For all other queries, respond normally.
        Always maintain context from previous messages and use that information in your responses.
        If you're not sure about something, admit that you don't know or don't have that information."""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])

    chatbot_agent = create_openai_tools_agent(llm, chatbot_tools, chatbot_prompt)
    chatbot_executor = AgentExecutor(agent=chatbot_agent, tools=chatbot_tools, memory=memory, verbose=True)

    print("Chatbot: Hello! I'm a chatbot that can also do mathematical calculations. How can I assist you today?")

    while True:
        user_input = input("You: ")
        keywords = extract_keywords(user_input)
        print("Keywords:", keywords)
        if (len(keywords) >= 1):
          for keyword in keywords:
              wikipedia_usage = wiki(keyword)
              memory.save_context({"user_input": user_input}, {"wikipedia_response": wikipedia_usage})

        result = chatbot_executor.invoke({"input": user_input})

        bot_response = result['output']
        print("Bot:", bot_response)

chatbot_updated_wiki()

Chatbot: Hello! I'm a chatbot that can also do mathematical calculations. How can I assist you today?
You: what is python
Keywords: ['python']


/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')




> Entering new AgentExecutor chain...
Python can refer to two different things:

1. Python (programming language): Python is a high-level, general-purpose programming language known for its readability and versatility. It supports multiple programming paradigms and has a comprehensive standard library. Python was created by Guido van Rossum and first released in 1991. It is widely used in various fields, including web development, data science, and machine learning.

2. Monty Python: Monty Python is a British comedy troupe formed in 1969, consisting of Graham Chapman, John Cleese, Terry Gilliam, Eric Idle, Terry Jones, and Michael Palin. They are known for their sketch comedy series "Monty Python's Flying Circus" and iconic films like "Monty Python and the Holy Grail" and "Life of Brian." Monty Python's work has had a significant influence on comedy and is considered groundbreaking in the world of television comedy.

> Finished chain.
Bot: Python can refer to two different things:

1

KeyboardInterrupt: Interrupted by user